# Hopfield Network Attractor Project

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Basic classes, util functions, activation functions

In [ ]:
# Util functions
def show_letter(pattern, ax = None):
    if ax == None:
      
        f, ax = plt.subplots(1, 1, figsize=(4, 4))
        f.tight_layout()
    side_len = int( pattern.size ** 0.5 + 0.5)
    ax.imshow(pattern.reshape(side_len, side_len), cmap='bone_r')
    ax.set_axis_off()

def add_noise(x_, noise_level=.2):
    noise = np.random.choice(
        [1, -1], size=len(x_), p=[1-noise_level, noise_level])
    return x_ * noise

In [ ]:
## Activation Functions

def ReLU(ws):
    data = [max(0,value) for value in ws]
    return np.array(data, dtype=float)

## for a single element
def leakyRelu(ws):
    if ws>0:
        return ws
    else:
        return 0.01*ws

## for a matrix
def leaky_relu(ws):
    alpha = 0.1
    return np.maximum(alpha*ws, ws)

### Update Rules

In [ ]:
#storkey learning rule
def storkey(self):
    self.W = np.zeros([self.num_neurons, self.num_neurons])

    for image_vector, _ in self.train_dataset:
        self.W += np.outer(image_vector, image_vector) / self.num_neurons
        net = np.dot(self.W, image_vector)

        pre = np.outer(image_vector, net)
        post = np.outer(net, image_vector)

        self.W -= np.add(pre, post) / self.num_neurons
    np.fill_diagonal(self.W, 0)

    #Pseudo-Inverse Rule (implemented in C++)
    # if (method == LearningMethod::PSEUDO_INVERSE)
    # {
    #     arma::mat w(N, N);
    #     int m = samples.size(); // št. učnih vzorcev
    #     for (int i = 0; i != N; i++)
    #     {
    #         for (int j = 0; j != N; j++)
    #         {
    #             w(i, j) = 0.0;
    #             for (int v = 0; v != m; v++)
    #             {
    #                 for (int u = 0; u != m; u++)
    #                 {
    #                     w(i, j) += samples[v][i] * (1.0 / Q(u, v, samples)) * samples[u][j];
    #                 }
    #             }
    #             w(i, j) = w(i, j) / N;
    #         }
    #     }
    #     std::cout << w << endl;

    #     // prekopiraj iz Armadillo mat objekta v 2D array
    #     for (int i = 0; i != N; i++)
    #     {
    #         for (int j = 0; j != N; j++)
    #         {
    #             W[i][j] = w(i, j);
    #         }
    #     }
    # }
def extended_storkey_update(x, weights):
    """
    Create an Op that performs a step of the Extended
    Storkey Learning Rule.
    Args:
      sample: a 1-D x Tensor of dtype tf.bool.
      weights: the weight matrix to update.
    Returns:
      An Op that updates the weights based on the sample.
    """
    scale = 1 / int(weights.get_shape()[0])
    numerics = 2*tf.cast(sample, weights.dtype) - 1
    row_sample = tf.expand_dims(numerics, axis=0)
    row_h = tf.matmul(row_sample, weights)

    pos_term = (tf.matmul(tf.transpose(row_sample), row_sample) +
                tf.matmul(tf.transpose(row_h), row_h))
    neg_term = (tf.matmul(tf.transpose(row_sample), row_h) +
                tf.matmul(tf.transpose(row_h), row_sample))
    return tf.assign_add(weights, scale * (pos_term - neg_term))

### The Main Class

In [ ]:
class HopfieldNetwork(object):
    def __init__(self, **kwargs):
      pass

    ## Training Step Options
    # ----------------------

    def TS_hebbian(self, training_list):
        """
        Apply the hebbian training algorhithm 
        with training_list as the input
        """
        m, n_units = np.shape(training_list)
        self.m = m
        self.n_units = n_units
        self.training_list = training_list
        self.weights = np.zeros((n_units, n_units))

        # Memory lossiness warning
        if n_units*0.14 < m:
            print("The number of memory patterns to be stored is > 14%% " +
                "of the model size. This may lead to problems." +
                "ref: https://doi.org/10.3389/fncom.2016.00144")

        # Hebbian rule
        for x in training_list:
            self.weights += np.outer(x, x) / m
        self.weights[np.diag_indices(n_units)] = 0
    
    def TS_storkey(self, training_list):
        # TODO: check if this works?
        m, num_neurons = np.shape(training_list)
        self.m = m
        self.num_neurons = num_neurons
        self.training_list = training_list
        self.weights = np.zeros([self.num_neurons, self.num_neurons])

        for image_vector in self.training_list:
            self.weights += np.outer(image_vector, image_vector) / self.num_neurons
            net = np.dot(self.weights, image_vector)

            pre = np.outer(image_vector, net)
            post = np.outer(net, image_vector)

            self.weights -= np.add(pre, post) / self.num_neurons
        np.fill_diagonal(self.weights, 0)


    # Inference Step Options
    # ----------------------

    def IS_tanh_threshold(self, X, N, gradient=1000, threshold=1):
        """
        Run the inference step N times starting with the input X0
        The activation function is tanh(a*x + b)
        Set gradient to 1 for normal tanh
        """
        # set up empty history
        Xs = np.zeros((N, len(X)))

        for i in range(N):
            # weighted sums
            ws = np.dot(X, self.weights)

            # activation function
            X = np.tanh(gradient * (ws - threshold))

            # check if there's change from previous entry
            if i > 0:
                if self._calculate_error(Xs[i-1], X) == 0:
                    Xs = Xs[:i]
                    # print(f"quit after {i} steps: steady state reached")
                    break

            # add entry to state history
            Xs[i] = X.copy()

        self.inference_history = Xs
        return Xs
    

    ## Evaluation functions
    # ---------------------

    def energy(self):
        """sum of values >= 0 over the inference history"""
        return np.sum([self.inference_history >= 0])
    
    def time(self):
        return len(self.inference_history)

    def perf(self, test):
        """difference between chosen input"""
        return self._calculate_error(test, self.inference_history[-1])

    def best_fit(self, y_hat):
        X_predict = self.inference_history[-1]
        score = self._validate(X_predict, y_hat)
        # TODO:
        return score

    def _validate(self, X_predict, y_hat):
        '''
        Return value: (is_correct, mse_to_target)
            is_correct: 0 or 1
            mse_target: Scalar
        '''
        # find the most similar picture in the dataset
        min_error = 1e10
        min_error_idx = -1
        for y_idx in range(self.m):
            # print(X_predict, self.training_list[y_idx])
            this_error = self._calculate_error(X_predict, self.training_list[y_idx])
            # print(f"idx={y_idx}, error={this_error}")
            if this_error < min_error:
                min_error = this_error
                min_error_idx = y_idx
        # print(min_error, min_error_idx)
        return (min_error_idx == y_hat), self._calculate_error(X_predict, self.training_list[y_hat])

    @staticmethod
    def _calculate_error(x1, x2):
        return np.sum(np.abs(x1 - x2))


class PerformanceMetric(object):
    def __init__(self):
        self.time = []
        self.energy = []
        self.is_correct = []
        self.error = []


# Dataset Preparation

### Dataset 2 (Demo, whole letters)

In [ ]:

import imageio as iio
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')

# read an image
img = iio.imread("/content/drive/MyDrive/Attractor Network Project/Images/pixel-arial-11-font-character-map.png")
# print(img.shape)
plt.imshow(img)

def generate_all_letter_dataset(new_size = 64):
    pixel_i = 115
    pixel_j = 110
    all_letter_final = np.empty((9 * 9, new_size, new_size), dtype="int8")
    for i in range(9):
        for j in range(9):
            # print(i, j)
            i_offset = 0
            if i >= 3:
                i_offset = 40
            if i >= 6:
                i_offset = 80

            subimg = img[i_offset + pixel_i * i : i_offset +pixel_i * (i+1),
                         pixel_j * j : pixel_j * (j+1),
                         :4].astype("uint8")
            new_img = Image.fromarray(subimg)

            new_img = new_img.resize((new_size, new_size))
            new_arr = np.asarray(new_img)
            # print(new_arr[20:30, 20:30, 3])
            new_arr = (new_arr[:, :, 3] > 100) * 2 - 1
            # all_letter_final[i * 9 + j] = new_arr[:, :, 0]
            all_letter_final[i*9+j] = new_arr[:]
    all_letter_final = all_letter_final.reshape(all_letter_final.shape[0], -1)
    return all_letter_final

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Calculate all letters up to resolution 100

LETTERS_RESOLUTION = [generate_all_letter_dataset(new_size=1)]
for i in range(0, 100):
    LETTERS_RESOLUTION.append(generate_all_letter_dataset(new_size = i+1))

In [ ]:
# # Set the letter resolution

# all_letter_final = generate_all_letter_dataset(new_size = 6)
# # print(all_letter_final)
# print(all_letter_final.shape)
# for i in range(9):
#     for j in range(9):
#         arr = all_letter_final[9 * i + j]
#         ax = plt.subplot(9, 9, i * 9 + j + 1)
#         show_letter(arr, ax)

In [ ]:
LETTERS_RESOLUTION[10].shape
all_letter_final = LETTERS_RESOLUTION[10]
for i in range(9):
    for j in range(9):
        arr = all_letter_final[9 * i + j]
        ax = plt.subplot(9, 9, i * 9 + j + 1)
        show_letter(arr, ax)

### Dataset 3 (MNIST)

In [ ]:
from sklearn.datasets import fetch_openml

# GET mnist data
mnist = fetch_openml(name='mnist_784', as_frame = False)
mnist_X = mnist.data
mnist_X.shape

import matplotlib.pyplot as plt
show_first_n = 10
for i in range(show_first_n):
  plt.subplot(1, show_first_n, i +1)
  plt.imshow(mnist_X[i].reshape(28, 28))

### Dataset 4(Demyan)

In [ ]:
import imageio as iio
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# all_images_dem = np.empty((6, 200 * 200))
all_images_dem = np.empty((6, 50 * 50))
for i in range(6):
  img = iio.imread(f"/content/drive/MyDrive/Attractor Network Project/Images/Hand Drawn/dem_{i+1}.jpg")
  print(img.shape)
  new_img = img.copy().astype("int")
  new_img[new_img < 100] = -1
  new_img[new_img >= 100] = 1
  new_img = new_img[::4, ::4]
  all_images_dem[i] = new_img.reshape(-1).copy()

# img = iio.imread(f"/content/drive/MyDrive/Attractor Network Project/Images/Hand Drawn/dem_1.jpg")
# all_images_dem.shape
# plt.hist(img.reshape(-1))
# img_set = set(tuple(img.reshape(-1).tolist()))
# print(img_set)

In [ ]:
fig, axs = plt.subplots(1, 6)
for i in range(6):
  show_letter(all_images_dem[i], axs[i])


# Training and validation process

### Demo for dataset 2

In [ ]:
# Setup and train
np.random.seed(0)
letter_index = [0,1 ,2,3]
letter_res = 4

all_letter_dataset = LETTERS_RESOLUTION[letter_res][letter_index]

all_letter_dataset = np.random.randint(0, 1, size = )
n_image, image_size = all_letter_dataset.shape
print(f"Num of images = {n_image}")

hop_net = HopfieldNetwork()
hop_net.TS_hebbian(all_letter_dataset)

# alphabet preview
letter_preview = LETTERS_RESOLUTION[letter_res]
print(letter_preview.shape)
for i in range(9):
    for j in range(9):
        arr = letter_preview[9 * i + j]
        ax = plt.subplot(9, 9, i * 9 + j + 1)
        show_letter(arr, ax)

In [ ]:
for (i, j) in [(0, 1), (0, 2), (0, 3)]:
  print(i, j, np.corrcoef(all_letter_dataset[i], all_letter_dataset[j])[0][1])



In [ ]:
# TODO: try to implement this thing where it shows the letters we're using?
# I duno how to do it in one plot...

# Show chosen letters
# for i, ltr in enumerate(all_letter_dataset):
#     ax = plt.subplot(2, graph_len, i+1)
#     show_letter(ltr, ax)

# Run the inference

noise_level = .0
# n_test_samples = 1000
n_test_samples = 10
n_iter = 100
show_last_n_letters = 3
for i in range(n_test_samples):
    # pick random image
    letter_idx = np.random.randint(0, n_image)

    # set custom letter index
    # letter_idx = 1

    # add noise
    x_test = all_letter_dataset[letter_idx].copy()
    x_test = add_noise(x_test, noise_level=noise_level)
    Xs = hop_net.IS_tanh_threshold(x_test, N=n_iter)

    # plot input
    ax = plt.subplot(n_test_samples, 2, i*2+1)
    show_letter(x_test, ax)
    ax = plt.subplot(n_test_samples, 2, i*2+2)
    show_letter(Xs[-1], ax)

    # # plot last n entries
    # for i, X in enumerate(Xs[-3:]):
    #     ax = plt.subplot(1, show_last_n_letters + 2, i+3)
    #     show_letter(X, ax)
        
    # TODO: evaluate the function

### A demo for validation

In [ ]:
# np.random.seed(0)

# all_letter_dataset = generate_all_letter_dataset(new_size = 150)
# all_letter_dataset = all_letter_dataset[[10, 20, 30, 5]]
# # all_letter_dataset = all_letter_dataset[::-1]
# n_image, image_size = all_letter_dataset.shape
# print(f"Num of images = {n_image}")

# hop_net = HopfieldNetwork()
# hop_net.TS_hebbian(all_letter_dataset)

# noise_level = .20
# # n_test_samples = 1000
# n_test_samples = 1
# n_iter = 1000
# for i in range(n_test_samples):
#     # add noise
#     # random_idx = np.random.randint(0, n_image - 1)
#     random_idx = 3
#     print(random_idx)
#     x_test = all_letter_dataset[random_idx].copy()
#     x_test = add_noise(x_test, noise_level=noise_level)
#     is_correct, error = hop_net.score(x_test, random_idx)
#     print(is_correct, error)
#     if i == 0:
#       fig, axs = plt.subplots(1, 6)
#       show_letter(all_letter_dataset[0], axs[0])
#       show_letter(all_letter_dataset[1], axs[1])
#       show_letter(all_letter_dataset[2], axs[2])
#       show_letter(all_letter_dataset[3], axs[3])
#       show_letter(x_test, axs[4])
#       show_letter(hop_net.IS_tanh_threshold(x_test)[-1], axs[5])
   

### Demo for dataset 3

In [ ]:
mnist_X[0].min(),mnist_X[0].max()

In [ ]:
n_images = 10
all_letter_dataset = mnist_X[:n_images].copy()
all_letter_dataset[all_letter_dataset < 10] = -1
all_letter_dataset[all_letter_dataset >= 10] = 1

for i in range(n_images):
  ax = plt.subplot(1, n_images, i+1)
  show_letter(all_letter_dataset[i], ax)


In [ ]:
hop_net = HopfieldNetwork()
hop_net.TS_hebbian(all_letter_dataset)

noise_level = .20
# n_test_samples = 1000
n_test_samples = 50
n_iter = 1000
for i in range(n_test_samples):
    # add noise
    random_idx = np.random.randint(0, n_images)
    # random_idx = 3
    x_test = all_letter_dataset[random_idx].copy()
    x_test = add_noise(x_test, noise_level=noise_level)
    Xs = hop_net.IS_tanh_threshold(x_test, 10)
    is_correct, error = hop_net._validate(x_test, random_idx)
    print(random_idx, is_correct, error)

    ax = plt.subplot(n_test_samples, 2, i * 2 +1)
    show_letter(x_test, ax)
    ax = plt.subplot(n_test_samples, 2, i * 2 +2)
    show_letter(Xs[-1], ax)


In [ ]:
import seaborn as sns
sns.heatmap(hop_net.weights)

In [ ]:
sns.heatmap(hop_net.weights[:100, :100])

### Demo for Dataset 4

In [ ]:
# all_images_dem
hop_net = HopfieldNetwork()
hop_net.TS_storkey(all_images_dem)

noise_level = .0
# n_test_samples = 1000
n_test_samples = 1
n_iter = 1000
n_images = len(all_images_dem)
for i in range(n_test_samples):
    # add noise
    # random_idx = np.random.randint(0, n_images)
    random_idx = 2
    x_test = all_images_dem[random_idx].copy()
    x_test = add_noise(x_test, noise_level=noise_level)
    Xs = hop_net.IS_tanh_threshold(x_test, 10)
    is_correct, error = hop_net._validate(x_test, random_idx)
    print(random_idx, is_correct, error)

    ax = plt.subplot(n_test_samples, 2, i * 2 + 1)
    show_letter(x_test, ax)
    ax = plt.subplot(n_test_samples, 2, i * 2 + 2)
    show_letter(Xs[-1], ax)


# Let's do experiments

### Exp 1: only changing number of neurons

In [ ]:
res = 16
ninput = 4
noise_level = .20
# n_test_samples = 1000
n_test_samples = 10
n_iter = 4
show_last_n_letters = 3
np.random.seed(4)

def do_test(res, dataset, noise_level, n_test_samples, show_last_n_letters):
  ninput = dataset.shape[0]
  hop_net = HopfieldNetwork()
  hop_net.TS_hebbian(dataset)
  # for i in range(ninput):
  #   ax = plt.subplot(1, ninput, i+1)
  #   show_letter(dataset[i], ax)
  pm = PerformanceMetric()

  for i in range(n_test_samples):
    # pick random image
    letter_idx = np.random.randint(0, ninput - 1)
    # set custom letter index
    # letter_idx = 1

    # add noise
    x_test_raw = all_letter_dataset[letter_idx].copy()
    x_test = add_noise(x_test_raw, noise_level=noise_level)
    Xs = hop_net.IS_tanh_threshold(x_test, N=n_iter)
    
    is_correct, error = hop_net._validate(Xs[-1], letter_idx)
    # print(hop_net.time())
    # print(hop_net.energy())
    # print(is_correct, error)
    pm.time.append(hop_net.time())
    pm.energy.append(hop_net.energy())
    pm.is_correct.append(is_correct)
    pm.error.append(error)
  return pm
    
    # print(hop_net.perf())
    # # plot input
    # ax = plt.subplot(1, show_last_n_letters + 1, 1)
    # show_letter(x_test_raw, ax)

    # # plot last n entries
    # for i, X in enumerate(Xs[-3:]):
    #     ax = plt.subplot(1, show_last_n_letters + 1, i+2)
    #     show_letter(X, ax)

# this_letter_dataset = LETTERS_RESOLUTION[letter_res][range(0, ninput)]
all_letter_dataset = mnist_X[:ninput]
pm = do_test(res, all_letter_dataset, noise_level, n_test_samples, show_last_n_letters)

In [ ]:
# pms = []
# min_res, max_res = 1, 10
# n_test_samples = 30
# neurons = [x**2 for x in range(min_res, max_res)]
# np.random.seed(10)
# for res in range(min_res, max_res):
#   pm = do_test(res, ninput, noise_level, n_test_samples, show_last_n_letters)
#   pms.append(pm)

# Plots


In [ ]:
pms = [pm]
neurons = [28 * 28]
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
# 1
ax_num_acc = axs[0][0]
ax_num_acc.plot(neurons, [np.mean(pm.is_correct) for pm in pms])
ax_num_acc.set_xlabel('#Neurons')
ax_num_acc.set_ylabel("Accuracy")
ax_num_acc.set_xticks(neurons)
ax_num_acc.set_title(f"#Neuron v.s. Accuracy (#trial = {n_test_samples}, #input_sample={ninput})")


# ax timestep accuracy
ax_time_acc = axs[0][1]
ax_time_acc.scatter([np.mean(pm.time) for pm in pms], [np.mean(pm.is_correct) for pm in pms])
ax_time_acc.set_xlabel(f"Mean timestamp on {n_test_samples} trials")
ax_time_acc.set_ylabel("Accuracy")
ax_time_acc.set_title("Mean timestamp v.s. Accuracy")

# ax energy accuracy
ax_energy_acc = axs[1][0]
ax_energy_acc.scatter([np.mean(pm.energy) for pm in pms], [np.mean(pm.is_correct) for pm in pms])
ax_energy_acc.set_xlabel(f"Mean energy on {n_test_samples} trials")
ax_energy_acc.set_ylabel("Accuracy")
ax_energy_acc.set_title("Mean energy v.s. Accuracy")

# ax energy accuracy
ax_energy_time = axs[1][1]
ax_energy_time.scatter([np.mean(pm.energy) for pm in pms], [np.mean(pm.time) for pm in pms])
ax_energy_time.set_xlabel("Mean energy")
ax_energy_time.set_ylabel("Mean Timestamp")
ax_energy_time.set_title("Mean energy v.s. Mean Timestamp")


### Exp2: Change the learning rule / activation functin

# Plotting Template

In [ ]:
# neurons = [25, 36, 49, 64, 81]
# accuracy = [0.5, 0.6, 0.7, 0.8, 0.9]
# time = [10, 50, 30, 10, 20]

# fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
# ax1.plot(neurons, accuracy)
# ax1.set_xlabel("#Neurons")
# ax1.set_ylabel("Accuracy")
# ax1.set_ylim(0, 1)

# ax2.plot(neurons, time)
# ax2.set_xlabel("#Neurons")
# ax2.set_ylabel("Time")

# ax3.plot(time, accuracy)
# ax3.set_xlabel("Time")
# ax3.set_ylabel("Accuracy")
# ax3.set_ylim(0, 1)